# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [2]:
import nuclio

## Environment

In [3]:
%nuclio cmd -c python -m pip install git+https://github.com/v3io/grafwiz.git

## Function

In [4]:
# nuclio: start-code

In [5]:
from grafwiz import *
import os

In [34]:
def deploy_dashboard(context, 
                     grafana_url:str = 'http://grafana', 
                     streamview_url:str = 'http://192.168.224.209:30100',
                     v3io_container:str = 'bigdata',
                     stocks_kv_table:str = 'stocks/stocks_kv',
                     stocks_tsdb_table:str = 'stocks/stocks_tsdb',
                     stocks_sentiment_tsdb_table:str = 'stocks/stocks_sentiment_tsdb'):
    # Create a new Grafana dashboard
    dash = Dashboard("stocks", start='now-1d', dataSource='Iguazio')

    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks/stocks_kv;backend=kv;container=bigdata")

    # Create a table and log viewer in one row
    tbl = Table('tbl1',span=8).source(table=stocks_kv_table,fields=['symbol','volume', 'price','last_updated'],container=v3io_container)
    log = Ajax(title='Log',url=streamview_url)
    dash.row([tbl,log])

    # Create 2 charts on the second row
    dash.row([Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']])
    dash.row([Graph('sentiment').series(table=stocks_sentiment_tsdb_table, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container)])
    
    # Deploy to Grafana
    return dash.deploy(grafana_url)

In [35]:
# nuclio: end-code

## Local test

In [36]:
from mlrun import code_to_function, run_local

fn = code_to_function('grafana_dashboard_deployer',
                      kind='job',
                      handler='deploy_dashboard',
                      project='stocks')
fn.spec.build.base_image = 'mlrun/mlrun'
fn.export('06-grafana.yaml')

> 2020-09-23 13:00:11,565 [info] function spec saved to path: 06-grafana.yaml


In [37]:
run_local(handler=deploy_dashboard)

> 2020-09-23 13:00:11,579 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2020-09-23 13:00:11,586 [info] starting run mlrun-664e71-deploy_dashboard uid=3c4361269ddd49dcacc6e85a7c1f1f7f  -> http://mlrun-api:8080
Dashboard stocks created successfully


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...7c1f1f7f,0,Sep 23 13:00:11,completed,mlrun-664e71-deploy_dashboard,v3io_user=adminkind=handlerowner=adminhost=jupyter-dcfbc696d-s229k,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3c4361269ddd49dcacc6e85a7c1f1f7f --project default , !mlrun logs 3c4361269ddd49dcacc6e85a7c1f1f7f --project default
> 2020-09-23 13:00:11,760 [info] run executed, status=completed
